In [103]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss


In [104]:
df = pd.read_pickle("/Users/simongutwein/mnt/qb/baumgartner/sgutwein84/test_case_results.pkl")
df.loc[df.model == "mixed_trained_UNET_1183.pt", "model"] = "B"
df.loc[df.model == "prostate_trained_UNET_2234.pt", "model"] = "A"

cases = ["Prostate", "Mixed", "Lymphnodes"]

for case in cases:
    df_tmp = df[df['test_modularity'].str.match(case)]

    print(case, np.round(ss.wilcoxon(df_tmp[df_tmp['model'].str.match("A")]["gamma"], df_tmp[df_tmp['model'].str.match("B")]["gamma"])[1],5))



Prostate 1.0
Mixed 6e-05
Lymphnodes 0.00116


In [105]:
df_train = pd.read_excel("/Users/simongutwein/home/baumgartner/sgutwein84/container/segments.xlsx", index_col=0)
df_test = pd.read_excel("/Users/simongutwein/home/baumgartner/sgutwein84/container/segments_tests.xlsx", index_col=0)
df_train

,test_case,fieldsize
0,p0_0,tensor(66.3105)
1,p0_1,tensor(46.2990)
2,p0_10,tensor(26.0326)
3,p0_11,tensor(60.4884)
4,p0_12,tensor(53.2269)
...,...,...
3365,h9_5,tensor(108.5390)
3366,h9_6,tensor(88.4404)
3367,h9_7,tensor(6.1126)
3368,h9_8,tensor(25.9148)


In [106]:
df_train['fieldsize'] = df_train['fieldsize'].map(lambda x: x.lstrip('tensor(').rstrip(')'))
df_test['fieldsize'] = df_test['fieldsize'].map(lambda x: x.lstrip('tensor(').rstrip(')'))

In [116]:
cases = ["p", "h", "l", "m", "n"]
cases_num = [45,15,15,15,15]

np.array(df_train[df_train['test_case'].str.match("h")]["fieldsize"]).astype(float).mean()


In [150]:
cases_mix = [f"p{x}_" for x in range(10)]
df_mix = pd.DataFrame()

for case in cases_mix:
    df_mix = df_mix.append(df_train[df_train['test_case'].str.contains(case)], ignore_index=True)


infos = {
    "case": "p_mix",
    "cases_num": df_mix.count()[0],
    "mean_train": np.array(df_mix["fieldsize"]).astype(float).mean(),
    "std_train": np.array(df_mix["fieldsize"]).astype(float).std(),
}

print(infos)

{'case': 'p_mix', 'cases_num': 414, 'mean_train': 35.491454830917874, 'std_train': 18.01637546766946}


In [113]:
infos = []
for case, num in zip(cases, cases_num):

    if case == "p":
        
        infos.append({
            "case": case,
            "cases_num": num,
            "train": df_train[df_train['test_case'].str.match(case)].count()[0],
            "test": df_test[df_test['test_case'].str.match(case)].count()[0],
            "mean_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
            "mean_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
        })
    else:
        infos.append({
            "case": case,
            "cases_num": num,
            "train": df_train[df_train['test_case'].str.match(case)].count()[0],
            "test": df_test[df_test['test_case'].str.match(case)].count()[0],
            "mean_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "mean_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
            "std_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
            "std_test": np.array(df_test[df_test['test_case'].str.match(case)]["fieldsize"]).astype(float).std(),
        })

df = pd.DataFrame(infos)
df




<ipython-input-113-7a720ebbbe23>:22: RuntimeWarning: Mean of empty slice.
  "mean_train": np.array(df_train[df_train['test_case'].str.match(case)]["fieldsize"]).astype(float).mean(),
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:261: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:221: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/usr/local/lib/python3.9/site-packages/numpy/core/_methods.py:253: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


,case,cases_num,train,test,mean_train,std_train,mean_test,std_test
0,p,45,10,306,47.005090,17.162090,34.573556,15.860270
1,h,15,550,379,63.009452,50.474623,68.890691,53.561800
2,l,15,547,272,24.097560,18.362432,22.816142,14.475376
3,m,15,237,419,40.720054,28.480739,40.600317,37.980044
4,n,15,0,659,NaN,NaN,25.952593,25.598072
